In [19]:
import pandas as pd 
import glob

In [20]:
all_data = []

In [34]:
! ls -lh ../modeling/data/sentence-data-*

-rw-r--r--  1 alex  staff   8.9M Nov  1 23:07 ../modeling/data/sentence-data-ap.csv.gz
-rw-r--r--  1 alex  staff   144M Nov  1 23:07 ../modeling/data/sentence-data-bbc.csv.gz
-rw-r--r--  1 alex  staff   596M Nov  1 23:14 ../modeling/data/sentence-data-guardian.csv
-rw-r--r--  1 alex  staff   139M Nov  1 16:03 ../modeling/data/sentence-data-independent.csv
-rw-r--r--  1 alex  staff    24M Nov  1 23:08 ../modeling/data/sentence-data-nyt.csv.gz
-rw-r--r--  1 alex  staff   100M Nov  1 16:49 ../modeling/data/sentence-data-reuters.csv
-rw-r--r--  1 alex  staff   6.4K Oct 31 23:11 ../modeling/data/sentence-data-small.csv
-rw-r--r--  1 alex  staff    27M Nov  1 03:07 ../modeling/data/sentence-data-wp.csv


In [21]:
for file in filter(lambda x: 'small' not in x, glob.glob('../modeling/data/sentence-data-*')):
    print(file)
    source = file.split('/')[-1].split('-')[-1].split('.')[0]
    try:
        df = pd.read_csv(file)
        df['source'] = source
        all_data.append(df)
    except:
        print('failed on %s' % file)

../modeling/data/sentence-data-wp.csv
../modeling/data/sentence-data-bbc.csv.gz
../modeling/data/sentence-data-ap.csv.gz
../modeling/data/sentence-data-independent.csv
../modeling/data/sentence-data-nyt.csv.gz


In [23]:
all_data_df = pd.concat(all_data)

In [38]:
all_data_df

(51578234, 11)

In [24]:
import unidecode

In [48]:
all_data_df['sentence'] = (
    all_data_df['sentence']
    .fillna('')
    .apply(lambda x: unidecode.unidecode(x.replace('<p>', '').replace('</p>', '')))
)

In [65]:
all_data_df.to_csv('../modeling/data/training_data_full.csv')

In [25]:
doc_lengths = all_data_df.assign(c=1).groupby(['source', 'entry_id', 'version'])['c'].count()

In [26]:
all_data_df = all_data_df.set_index(['source', 'entry_id', 'version'])

In [27]:
empty_sents = (
    all_data_df
       .loc[lambda df: (df['sentence'] == '') | df['sentence'].isnull()]
       .index
       .drop_duplicates()
)
null_data = (
    all_data_df
    .loc[lambda df: df.isnull().any(axis=1)]
    .index
    .drop_duplicates()
)

In [166]:
all_data_df_small = (
    all_data_df
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(empty_sents)]
    .loc[lambda df: ~df.index.isin(null_data)]
)

In [115]:
all_data_df_small.to_csv('../modeling/data/training_data_short_15.csv')

In [ ]:
all_data_df_small = pd.read_csv('../modeling/data/training_data_short_15.csv.gz')

In [28]:
all_data_df_small_downsampled = all_data_df.loc[
    doc_lengths
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(empty_sents)]
    .loc[lambda df: ~df.index.isin(null_data)]
    .sample(150_000).index
]

In [29]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_150000.csv')

In [247]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_50000.csv')

In [248]:
all_data_df_small_downsampled = all_data_df.loc[
    doc_lengths
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(empty_sents)]
    .loc[lambda df: ~df.index.isin(null_data)]
    .sample(30_000).index
]

In [249]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_30000.csv')

In [250]:
all_data_df_small_downsampled = all_data_df.loc[
    doc_lengths
    .loc[doc_lengths.loc[lambda s: (s < 15) & (s > 5)].index]
    .loc[lambda df: ~df.index.isin(empty_sents)]
    .loc[lambda df: ~df.index.isin(null_data)]
    .sample(10_000).index
]

In [567]:
all_data_df_small_downsampled.to_csv('../modeling/data/training_data_short_15__sampled_10000.csv')

In [195]:
docs = all_data_df_small_downsampled.index.drop_duplicates()

In [199]:
for d in docs:
    doc = all_data_df_small_downsampled.loc[d]
    t = torch.tensor(doc[['deleted_label', 'unchanged_label', 'edited_label']].values)
    torch.where(t == 1)[1]

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  


# Make Document Data

In [1]:
import pandas as pd 

In [2]:
all_data_df_small_downsampled = pd.read_csv('../modeling/data/training_data_short_15__sampled_50000.csv', index_col=[0,1,2])

In [3]:
all_data_df_small_downsampled.groupby(level=[0,1,2])[['add_below_label', 'add_above_label']].sum().corr()

,add_below_label,add_above_label
add_below_label,1.000000,0.678597
add_above_label,0.678597,1.000000


In [30]:
import numpy as np 

In [31]:
def zero_or_pm_one(x):
    if x == 0:
        return 0
    elif x < 0:
        return -1
    elif x > 0:
        return 1

In [32]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

def applyParallel(dfGrouped, func, return_dict=False):
    ret_val = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in tqdm(dfGrouped))
    if return_dict:
        names = [name for name, group in dfGrouped]
        ret_val = dict(zip(names, ret_val))
    return ret_val

In [7]:
num_splits = 2
def calculate_doc_stats(df):
    return (        
        df
         .assign(section=lambda df: 
                    df['sent_idx']
                         .pipe(lambda s: s.clip(upper=int(len(s)/ num_splits) * num_splits - 1))
                         .pipe(lambda s: pd.cut(s, np.arange(0, len(s)+num_splits, int(len(s) / num_splits)), right=False))
            )
         .pipe(lambda df:
                pd.concat([
                    (
                    df
                     .set_index('section', append=True)
                     .groupby(level=[0,1,2,3])
                     [['deleted_label', 'add_above_label', 'add_below_label', 'edited_label', 'unchanged_label']]
                     .sum()
                ), 
                   (
                      df[['section', 'refactored_label']]
                         .assign(refactored_label=lambda df: df['refactored_label'].apply(zero_or_pm_one))
                          .rename(columns={'refactored_label': 'level_0'})
                         .set_index(['section', 'level_0'], append=True)
                         .groupby(level=[0,1,2,3,4]).size().unstack()
                         .rename(columns={-1:'refactored down', 0: 'not refactored', 1:'refactored up'})
                )], axis=1)
              )
         .reset_index('section', drop=True)
         .assign(group=lambda df: np.arange(len(df)))
         .assign(size=lambda df: len(df))
         .set_index(['group', 'size'], append=True)
         .unstack(level=[3,4])
    )
        
#  .assign(c=lambda df: np.arange(len(df)))
#  .assign(group=lambda df: df.groupby(level=[0,1,2])['c'].rank("dense"))
#  .assign(size=lambda df: df.groupby(level=[0,1,2])['c'].transform(lambda df: len(df)))
#  .set_index(['group', 'size'], append=True)
#  .drop('c',axis=1)
#  .unstack(level=[3,4]))

In [33]:
grouped = (all_data_df_small_downsampled
 .groupby(level=[0,1,2]))

In [ ]:
from tqdm.auto import tqdm
groups = applyParallel(grouped, calculate_doc_stats)

In [34]:
sentences = applyParallel(grouped, lambda s: '<SENT>'.join(s['sentence'].tolist()), return_dict=True)

  0%|          | 0/150000 [00:00<?, ?it/s]

In [12]:
doc_level_data_small = pd.concat(groups)

In [13]:
doc_level_data_small = doc_level_data_small.fillna(0)

In [14]:
doc_level_data_small.columns = list(map(lambda x: '%s, %s/%s'% x, doc_level_data_small.columns))

In [15]:
doc_level_data_small = pd.concat([
    doc_level_data_small, 
    pd.Series(sentences).to_frame('sentences')
], axis=1)

In [36]:
(doc_level_data_small
 .reset_index()
 .rename(columns={'level_0':  'source', 'level_1': 'entry_id', 'level_2': 'version'})
 .to_csv('../modeling/data/doc_training_data_short_15__sampled_30000.csv', index=False)
)

In [35]:
doc_level_data_not_expanded = (
    pd.concat([
    all_data_df_small_downsampled
        .groupby(level=[0,1,2]).sum().astype(int).drop('sent_idx', axis=1),
    pd.Series(sentences).to_frame('sentences')
    ], axis=1)
)

In [36]:
doc_level_data_not_expanded.shape

(150000, 7)

In [37]:
(
    doc_level_data_not_expanded
     .reset_index()
     .rename(columns={'level_0':  'source', 'level_1': 'entry_id', 'level_2': 'version'})
     .to_csv('../modeling/data/doc_training_data_short_15_not_expanded__sampled_150000.csv', index=False)
)

In [490]:
(        group
         .assign(section=lambda df: 
                    df['sent_idx']
                         .pipe(lambda s: s.clip(upper=int(len(s)/ num_splits) * num_splits - 1))
                         .pipe(lambda s: pd.cut(s, np.arange(0, len(s)+num_splits, int(len(s) / num_splits)), right=False))
            )
)

sent_idx  \
source entry_id version             
ap     27       5.0           0.0   
                5.0           1.0   
                5.0           2.0   
                5.0           3.0   
                5.0           4.0   
                5.0           5.0   

                                                                  sentence  \
source entry_id version                                                      
ap     27       5.0       MAIDUGURI, Nigeria (AP) -- Witnesses say Boko...   
                5.0       One witness, who spoke on condition of anonym...   
                5.0       The witness says eight Boko Haram fighters we...   
                5.0       On Tuesday, Nigeria's air force bombed the ca...   
                5.0       Nigerian officials have said it was an accident.   
                5.0       On Friday, medical charity Doctors Without Bo...   

                         deleted_label  add_above_label  add_below_label  \
source entry_id version                                                    
ap     27       5.0                0.0              0.0              0.0   
                5.0                1.0              0.0              0.0   
                5.0                0.0              0.0              1.0   
                5.0                0.0              1.0              1.0   
                5.0                0.0              1.0              0.0   
                5.0                0.0              0.0              6.0   

                         edited_label  unchanged_label  refactored_label  \
source entry_id version                                                    
ap     27       5.0               1.0              0.0               0.0   
                5.0               0.0              0.0               0.0   
                5.0               1.0              0.0               0.0   
                5.0               0.0              1.0               0.0   
                5.0               1.0              0.0               0.0   
                5.0               1.0              0.0               0.0   

                        section  
source entry_id version          
ap     27       5.0      [0, 3)  
                5.0      [0, 3)  
                5.0      [0, 3)  
                5.0      [3, 6)  
                5.0      [3, 6)  
                5.0      [3, 6)

In [234]:
t = (doc
 .assign(section=lambda df: 
            df['sent_idx']
                 .pipe(lambda s: s.clip(upper=int(len(s) / 3) * 3 - 1))
                 .pipe(lambda s: pd.cut(s, np.arange(0, len(s), int(len(s) / 3)), right=False))
    )
)

In [328]:
(t[['section', 'refactored_label']]
 .assign(refactored_label=lambda df: df['refactored_label'].apply(zero_or_pm_one))
  .rename(columns={'refactored_label': 'level_0'})
 .set_index(['section', 'level_0'], append=True)
 .groupby(level=[0,1,2,3,4]).size().unstack()
 .rename(columns={-1:'refactored down', 0: 'not refactored', 1:'refactored up'})
 .reset_index(drop=True)
 .unstack()
 .reset_index()
 .assign(label=lambda df: df.apply(lambda x: '%s, %s' % (x['level_0'], '%s/%s' % (x['level_1'] + 1, 3)), axis=1))
 .set_index('label')[0]
)

label
refactored down, 1/3    0
refactored down, 2/3    0
refactored down, 3/3    1
not refactored, 1/3     4
not refactored, 2/3     4
not refactored, 3/3     5
Name: 0, dtype: int64

In [346]:
(t
 .set_index('section', append=True)
 .groupby(level=[0,1,2,3])
 [['deleted_label', 'add_above_label', 'add_below_label', 'edited_label', 'unchanged_label']]
 .sum()
)

deleted_label  add_above_label  \
source entry_id version section                                   
nyt    706871   1.0     [0, 4)             0.0              0.0   
                        [4, 8)             0.0              0.0   
                        [8, 12)            0.0              0.0   

                                 add_below_label  edited_label  \
source entry_id version section                                  
nyt    706871   1.0     [0, 4)               0.0           0.0   
                        [4, 8)               0.0           0.0   
                        [8, 12)              0.0           2.0   

                                 unchanged_label  
source entry_id version section                   
nyt    706871   1.0     [0, 4)               4.0  
                        [4, 8)               4.0  
                        [8, 12)              4.0

In [372]:
import numpy as np 

In [391]:
def rename_cols(x):
    print(x)

In [398]:
list(map(lambda x: '%s, %s/%s' % (x[0], x[1], x[2]), t2.columns))

['deleted_label, 1.0/3',
 'deleted_label, 2.0/3',
 'deleted_label, 3.0/3',
 'add_above_label, 1.0/3',
 'add_above_label, 2.0/3',
 'add_above_label, 3.0/3',
 'add_below_label, 1.0/3',
 'add_below_label, 2.0/3',
 'add_below_label, 3.0/3',
 'edited_label, 1.0/3',
 'edited_label, 2.0/3',
 'edited_label, 3.0/3',
 'unchanged_label, 1.0/3',
 'unchanged_label, 2.0/3',
 'unchanged_label, 3.0/3',
 'refactored down, 1.0/3',
 'refactored down, 2.0/3',
 'refactored down, 3.0/3',
 'not refactored, 1.0/3',
 'not refactored, 2.0/3',
 'not refactored, 3.0/3']

In [424]:
(doc
 .assign(section=lambda df: 
            df['sent_idx']
                 .pipe(lambda s: s.clip(upper=int(len(s)/ 3) * 3-1))
                 .pipe(lambda s: pd.cut(s, np.arange(0, len(s), int(len(s)/3)), right=False))
    )
 .pipe(lambda df:
        pd.concat([
            (
            t
             .set_index('section', append=True)
             .groupby(level=[0,1,2,3])
             [['deleted_label', 'add_above_label', 'add_below_label', 'edited_label', 'unchanged_label']]
             .sum()
        ), 
           (
              t[['section', 'refactored_label']]
                 .assign(refactored_label=lambda df: df['refactored_label'].apply(zero_or_pm_one))
                  .rename(columns={'refactored_label': 'level_0'})
                 .set_index(['section', 'level_0'], append=True)
                 .groupby(level=[0,1,2,3,4]).size().unstack()
                 .rename(columns={-1:'refactored down', 0: 'not refactored', 1:'refactored up'})
        )], axis=1)
      )
 .reset_index(drop=True)
 .assign(group=lambda df: np.arange(len(df)))
 .assign(size=lambda df: len(df))
 .set_index(['group', 'size'])
 .unstack(level=[0,1])
)

                 group  size
deleted_label    0      3       0.0
                 1      3       0.0
                 2      3       0.0
add_above_label  0      3       0.0
                 1      3       0.0
                 2      3       0.0
add_below_label  0      3       0.0
                 1      3       0.0
                 2      3       0.0
edited_label     0      3       0.0
                 1      3       0.0
                 2      3       2.0
unchanged_label  0      3       4.0
                 1      3       4.0
                 2      3       4.0
refactored down  0      3       0.0
                 1      3       0.0
                 2      3       1.0
not refactored   0      3       4.0
                 1      3       4.0
                 2      3       5.0
dtype: float64

In [656]:
from abc import ABCMeta

###
### Mixer
# mixin handlers
class AutoMixinMeta(ABCMeta):
    """
        Helps us conditionally include Mixins, which is useful if we want to switch between different
        combinations of models (ex. SBERT with Doc Embedding, RoBERTa with positional embeddings).

        class Sub(metaclass = AutoMixinMeta):
            def __init__(self, name):
            self.name = name
    """

    def __call__(cls, *args, **kwargs):
        try:
            mixin = kwargs.pop('mixin')
            if isinstance(mixin, list):
                mixin_names = list(map(lambda x: x.__name__, mixin))
                mixin_name = '.'.join(mixin_names)
                cls_list = tuple(mixin + [cls])
            else:
                mixin_name = mixin.__name__
                cls_list = tuple([mixin, cls])

            name = "{}With{}".format(cls.__name__, mixin_name)
            cls = type(name, cls_list, dict(cls.__dict__))
        except KeyError:
            pass
        return type.__call__(cls, *args, **kwargs)


class Mixer(metaclass = AutoMixinMeta):
    """ Class to mix different elements in.

            model = Mixer(config=config, mixin=[SBERTMixin, BiLSTMMixin, TransformerBase])
    """
    pass

In [671]:
class A():
    pass

class B():
    pass

config={
    'test_a': True, 
    'test_b': True
}

def get_mixins(config):
    mixins = []
    if config['test_a']:
        mixins.append(A)
    if config['test_b']:
        mixins.append(B)
    return mixins

to_mix = get_mixins(config)

c = Mixer(mixin=to_mix)

import pickle
pickle.dump(c, open('tmp/test.pkl', 'wb'))

PicklingError: Can't pickle <class '__main__.MixerWithA.B'>: attribute lookup MixerWithA.B on __main__ failed